In [96]:
import seaborn as sns
import pandas as pd
import numpy as np
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [97]:
df['day'].unique()

[Sun, Sat, Thur, Fri]
Categories (4, object): [Sun, Sat, Thur, Fri]

In [98]:
df['time'].unique()

[Dinner, Lunch]
Categories (2, object): [Dinner, Lunch]

In [99]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [100]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [101]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): [Lunch, Dinner]

In [102]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [103]:
df.time.unique()

array([0, 1])

In [104]:
# independent and dependent feature
x=df.drop(labels=['time'],axis=1)
y=df.time

In [105]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [106]:
x['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [107]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [108]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [109]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## handle missing values
from sklearn.preprocessing import StandardScaler ##feature scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer 

In [110]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [111]:
## feature engineering automation
## numerical pipeline
num_pipeline=Pipeline(
       steps=[
           ('imputer',SimpleImputer(strategy='median')), ##missing values
           ('scaler',StandardScaler())
       ]
)
## categorical pipeline
cat_pipeline=Pipeline(
                  steps=[
                    ('imputer',SimpleImputer(strategy='most_frequent')),## handling missing values
                    ('onehotencoder',OneHotEncoder()) ## categorical features to numerical
                  ]
                 )

In [112]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [113]:
x_train=preprocessor.fit_transform(x_train)
x_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [114]:
x_test=preprocessor.fit_transform(x_test)
x_test

array([[ 0.19029449,  0.48117084, -0.55602186,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-1.00973989, -0.57426535, -0.55602186,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.70397437, -0.57426535,  1.46213155,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.8494989 ,  2.25215697,  1.46213155,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.55036021, -0.57426535, -0.55602186,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.01930427, -0.57426535, -0.55602186,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.   

In [119]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# module training automation
models={
      'Random forest':RandomForestClassifier(),
      'Logistic Regression':LogisticRegression(),
      'Decision Tree':DecisionTreeClassifier()
      }

In [120]:
def  evaluate_model(x_train,y_train,x_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        # train model
        model.fit(x_train,y_train)
        
        # predicting testing data
        y_test_pred=model.predict(x_test)
        
        # get accuracy for test data
        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=test_model_score
        
        return report
        
    
    

In [121]:
from sklearn.metrics import accuracy_score

In [122]:
 evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random forest': 0.9795918367346939}